# Open Jupyter Notebook in Google Colab

1. Go to the following URL:
https://colab.research.google.com/


2. Click on GitHub tab.

<img src="../assets/colab1.png" width="500" alt="Create API Key button"/>


3. Paste the URL of this repository:

https://github.com/oh-scipe/llm-workshop26/tree/main/tutorials/day1

<img src="../assets/colab2.png" width="500" alt="Create API Key button"/>

4. Select the notebook you want to open.

# Google Gemini

This workshop notebook covers setup, authentication, text generation, conversations, streaming, and handling different content types.

## What are Large Language Models (LLMs)?

**Large Language Models** are deep neural networks trained on vast amounts of text data to understand and generate human-like language. They learn statistical patterns, relationships, and structures in language through a process called **unsupervised learning**.

### Key Concepts in Language Modeling:

1. **Token-based Processing**: Text is broken into tokens (words, subwords, or characters), and the model learns to predict the next token given previous context.

2. **Transformers Architecture**: Modern LLMs use the transformer architecture, which employs attention mechanisms to understand relationships between tokens, regardless of their distance in text.

3. **Pre-training and Fine-tuning**: Models are first pre-trained on massive datasets, then fine-tuned for specific tasks or aligned with human preferences.

4. **Autoregressive Generation**: LLMs generate text one token at a time, using previously generated tokens as context for the next prediction.

Google Gemini represents the latest generation of multimodal LLMs, capable of understanding and generating not just text, but also images, audio, and video.

## 1. Install and Import Required Libraries

First, we'll install the Google GenAI SDK and import necessary libraries.

The latest recommended package is `google-genai` which is the official Google AI SDK for accessing Gemini models.

In [ ]:
# Install the Google GenAI SDK
# Uncomment the line below to install (if not already installed)
# !pip install -q -U google-genai

# Import required libraries for interacting with the Gemini API
import os       # For environment variable access
import json     # For handling JSON data
from google import genai  # Official Google Generative AI SDK

## 2. Set Up API Authentication

### Authentication in API-based LLMs

The Gemini API uses **API key authentication** to identify and authorize your requests. API keys serve multiple purposes:

1. **Identity Verification**: Confirms you're an authorized user
2. **Usage Tracking**: Monitors your API calls for billing and rate limiting
3. **Security**: Prevents unauthorized access to the service
4. **Resource Allocation**: Manages quotas and priorities

**Best Practices for API Key Security:**
- Store keys in environment variables, never hardcode them
- Use `.env` files for local development (add to `.gitignore`)
- Rotate keys periodically
- Use separate keys for development, testing, and production
- Revoke compromised keys immediately

### Getting Your Google Gemini API Key

Follow these steps to obtain your free API key:

**Step 1**: Go to Google AI Studio  
https://aistudio.google.com/app/api-keys

**Step 2**: Click on "Create API Key"  
<img src="../assets/gem1.png" width="250" alt="Create API Key button"/>

**Step 3**: Choose an arbitrary name and click "Create Key"  
<img src="../assets/gem2.png" width="250" alt="Name your API key"/>

**Step 4**: Copy your key securely  
<img src="../assets/gem3.png" width="250" alt="Copy API key"/>

⚠️ **Important**: Treat your API key like a password. Never share it or commit it to version control.

In [ ]:
# Paste your API key here (get it from https://aistudio.google.com/app/api-keys)
api_key = ""

# Validate that API key is set before proceeding
if not api_key:
    raise ValueError(
        "API key not found! Please set your Gemini API key.\n"
        "Get your key at: https://aistudio.google.com/app/api-keys\n"
        "Then set it in the code above or use: export GEMINI_API_KEY='your-api-key'"
    )

# Display masked API key for verification (shows only first 4 and last 4 characters)
print(f"API Key found: {api_key[:4]}****{api_key[-4:]}")

client = genai.Client(api_key=api_key) # Initialize the Gemini client with your API key

## 3. Available Gemini Models

Let's explore the available models and their capabilities.

In [ ]:
try:
    # Fetch list of all available Gemini models from the API
    models_pager = client.models.list()
    
    # Print header for the model list
    print("Available Gemini Models:\n")
    print(f"{'Model Name':<40} {'Description':<50}")
    print("-" * 90)
    
    # Iterate through each model and display its information
    count = 0
    for model in models_pager:
        # Extract just the model name (remove the 'models/' prefix)
        model_name = model.name.split('/')[-1]
        
        # Determine model type based on display name
        desc = "Text Generation Model"
        if "vision" in model.display_name.lower():
            desc = "Multimodal Model (Text, Image)"
        elif "flash" in model.display_name.lower():
            desc = "Fast, Efficient Model"
        
        print(f"{model_name:<40} {desc:<50}")
        count += 1
        
except Exception as e:
    # Handle errors gracefully with helpful troubleshooting information
    error_msg = str(e)
    print(f"✗ Error listing models: {error_msg[:200]}")
    print("\nPossible issues:")
    print("- Invalid API key")    
    print("- Network connectivity problems")    
    print("- API service temporarily unavailable")

### 3.1 Available Gemini Models (Based on Your API Access)

The cell above shows the models currently available in your account. Here's a breakdown of the main models you have access to:

| Model Name | Type | Best Use Case |
|------------|------|---------------|
| **gemini-3-pro-preview** | Preview/Experimental | Latest Gemini 3.0 Pro - complex reasoning, analysis |
| **gemini-3-flash-preview** | Preview/Experimental | Latest Gemini 3.0 Flash - fast, efficient, high-volume tasks |
| **gemini-3-pro-image-preview** | Multimodal Preview | Image generation and understanding |
| **gemma-3-27b-it** | Open Model | Instruction-tuned Gemma model (27B parameters) |
| **deep-research-pro-preview-12-2025** | Specialized | Advanced research and analysis tasks |

### Model Naming Convention:

- **Preview/Experimental** models: Latest features, may change before stable release
- **Flash** variants: Optimized for speed and cost-effectiveness
- **Pro** variants: Balanced performance for complex tasks
- **Image** variants: Support multimodal input/output (images + text)
- **Deep Research**: Specialized for in-depth analysis and research tasks
- **Gemma**: Google's open-source model family (different from Gemini)

### Choosing the Right Model:

1. **Fast responses & high volume** → Use `gemini-3-flash-preview`
2. **Complex reasoning & analysis** → Use `gemini-3-pro-preview`
3. **Image generation/understanding** → Use `gemini-3-pro-image-preview`
4. **Research tasks** → Use `deep-research-pro-preview-12-2025`
5. **Open-source option** → Use `gemma-3-27b-it`

**Important**: Preview models may change or be deprecated. Check the official documentation for the latest stable releases.

**Resources:**
- **Rate Limits**: https://ai.google.dev/gemini-api/docs/rate-limits  
- **Pricing**: https://ai.google.dev/gemini-api/docs/pricing
- **Model Documentation**: https://ai.google.dev/gemini-api/docs/models

## 4. Generate Text Responses

The most basic use of the Gemini API: sending a prompt and getting a text response.

### Language Modeling Fundamentals: Text Generation

At its core, **text generation** is the process of predicting the next token given a sequence of previous tokens. This is formalized as:

$$P(w_t | w_1, w_2, ..., w_{t-1})$$

Where $w_t$ is the token at position $t$, and the model computes the probability distribution over all possible next tokens.

**Key Parameters in Generation:**

1. **Temperature** ($\tau$): Controls randomness in token selection. Lower values make the model more deterministic.
   $$P_i = \frac{\exp(z_i / \tau)}{\sum_j \exp(z_j / \tau)}$$

2. **Top-k Sampling**: Limits selection to the k most probable tokens, preventing unlikely options.

3. **Top-p (Nucleus) Sampling**: Selects from the smallest set of tokens whose cumulative probability exceeds p.

Let's see this in action with a simple text generation example:

In [ ]:
# Select which model to use for text generation
model = "gemini-3-flash-preview"

# Define the prompt/question to send to the model
prompt = "Explain quantum computing in 2-3 sentences for someone who is new to the topic."
    
# Display what we're sending to the model
print("\033[1m\033[4mModel:\033[0m", model)
print("\033[1m\033[4mPrompt:\033[0m", prompt)
print()

# Generate content using the selected model and prompt
response = client.models.generate_content(
    model=model,
    contents=prompt
)

# Extract the response parts from the first candidate
parts = response.candidates[0].content.parts

# Combine all text parts into a single output string
text_out = "".join(p.text for p in parts if getattr(p, "text", None))

print("\033[1m\033[4mResponse:\033[0m")
print(text_out)

### 4.1 Generation Parameters

You can customize the model's behavior using generation parameters.

### Language Modeling: Controlling Generation with Parameters

The way an LLM generates text can be dramatically influenced by **generation parameters**. Understanding these parameters is crucial for controlling the model's creativity, coherence, and reliability.

**Temperature**: Think of temperature as a "creativity knob"
- Low temperature (0.0-0.3): Deterministic, focused on most likely tokens → Good for factual tasks
- Medium temperature (0.4-0.7): Balanced creativity → Good for general conversation
- High temperature (0.8-1.0+): More random, exploratory → Good for creative writing

**Top-k Sampling**: Restricts the model to choosing from only the k most likely next tokens. This prevents the model from selecting improbable words while maintaining diversity.

**Top-p (Nucleus) Sampling**: Dynamically adjusts the number of tokens considered by selecting from the smallest set whose cumulative probability exceeds p. This is often more effective than top-k.

**Max Output Tokens**: Limits the length of the response, helping to control costs and response time.

**Thinking Config**: Controls the model's extended reasoning behavior. Setting `thinking_budget=0` disables the model's internal thinking process, which speeds up responses and reduces token usage for tasks that don't require complex reasoning.

Let's experiment with different parameters:

In [ ]:
from google.genai import types

model = "gemini-2.5-flash"  # Using a cheaper yet fast model

thinking_config=types.ThinkingConfig(thinking_budget=0) # Disable thinking for faster responses

# Using generation configuration to customize behavior
gen_config = types.GenerateContentConfig(
    temperature=0.7,  # 0.0 = deterministic, 1.0 = more random
    # top_p=0.95,       # Nucleus sampling parameter
    top_k=40,         # Top K sampling parameter
    max_output_tokens=512,  # Limit output length
    thinking_config=thinking_config,
)

prompt = "Write a creative poem about artificial intelligence."

response = client.models.generate_content(
    model=model,
    contents=prompt,
    config=gen_config
)

print("\033[1m\033[4mResponse:\033[0m")
print(response.text)

## 5. Use System Instructions

System instructions allow you to set the model's behavior, tone, and constraints for all requests in a conversation.

### Language Modeling: System Instructions as Context Priming

**System instructions** are a powerful technique in modern LLMs that leverages the concept of **prompt engineering** and **in-context learning**.

**How It Works:**
- System instructions are prepended to every user message in the conversation
- They set the "persona" or "role" of the model, constraining its behavior
- The model uses this context to adjust its response style, tone, and content

**Why It Matters:**
In transformer models, the attention mechanism allows the model to reference the system instruction when generating each token. This creates a form of **soft conditioning** where the instruction guides generation without explicit fine-tuning.

**Best Practices for System Instructions:**
1. Be specific and clear about the desired behavior
2. Include formatting requirements if needed
3. Specify constraints (e.g., language level, tone, length)
4. Define what the model should and shouldn't do

System instructions are especially useful for:
- Role-playing scenarios (tutors, assistants, experts)
- Consistent formatting of outputs
- Domain-specific behavior
- Safety and content filtering

In [ ]:
# Select the model to use
model = "gemini-2.5-flash"

# Define system instructions to set the model's persona and behavior
# This acts as a persistent context that influences all responses
system_instruction = """You are a helpful Python programming tutor. 
- Provide clear, beginner-friendly explanations
- Always include code examples
- Encourage questions and practice
- Use simple language, avoid jargon"""

# User's question
prompt = "How do I read a file in Python?"

# Generate response with system instruction applied
response = client.models.generate_content(
    model=model,
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction,  # Apply the tutor persona
        max_output_tokens=300,                   # Limit response length
        thinking_config=thinking_config,         # Disable thinking tokens
    )
)

# Display the tutor's response
print("\033[1m\033[4mSystem Instruction:\033[0m Python Programming Tutor")
print("\033[1m\033[4mPrompt:\033[0m", prompt)
print()
print("\033[1m\033[4mResponse:\033[0m")
print(response.text)

## 6. Multi-turn Conversations

Use the ChatSession class to build interactive conversations that maintain context across multiple turns.

### Language Modeling: Context Windows and Conversation History

Multi-turn conversations demonstrate a critical concept in LLMs: **context maintenance** and the **attention mechanism**.

**Context Window**: The maximum number of tokens (both input and output) that a model can consider at once. Gemini models have context windows ranging from 1M to 2M tokens.

**How Conversations Work:**
1. Each message (user and assistant) is stored in the conversation history
2. When generating a response, the model attends to all previous messages
3. The **self-attention mechanism** computes relevance scores between the current token being generated and all previous tokens
4. This allows the model to maintain coherence across multiple turns

**Mathematical Foundation:**
For each position in the sequence, attention is computed as:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where Q (query), K (key), and V (value) are learned projections of the input embeddings.

**Challenges:**
- **Context length limitations**: Older models could only handle short conversations
- **Attention complexity**: Grows quadratically with sequence length ($O(n^2)$)
- **Context management**: Deciding what to keep when approaching limits

Let's see how the model maintains context across multiple conversation turns:

In [ ]:
# Select the model for the chat session
model = "gemini-2.5-flash"

# Configure the chat session with system instructions and parameters
chat_config = types.GenerateContentConfig(
    system_instruction="You are a friendly and knowledgeable assistant about machine learning.",
    temperature=0.7,                # Balanced creativity for conversation
    max_output_tokens=500,          # Allow longer responses
    thinking_config=thinking_config,  # Disable thinking tokens
)

# Initialize a stateful chat session that maintains conversation history
chat = client.chats.create(
    model=model,
    config=chat_config
)

try:
    # First conversation turn: Ask a basic question
    message1 = "What is machine learning?"
    print("="*70)
    print("\033[1m\033[4mTURN 1\033[0m")
    print("="*70)
    print("\033[1mUser:\033[0m", message1)
    response1 = chat.send_message(message1)
    print("\033[1mAssistant:\033[0m", response1.text)
    print()
    
    # Second turn: Follow-up question (context from turn 1 is maintained)
    message2 = "Can you give me a practical example?"
    print("="*70)
    print("\033[1m\033[4mTURN 2\033[0m")
    print("="*70)
    print("\033[1mUser:\033[0m", message2)
    response2 = chat.send_message(message2)
    print("\033[1mAssistant:\033[0m", response2.text)
    print()
    
    # Third turn: More specific follow-up (builds on entire conversation)
    message3 = "How would you apply this to predicting house prices?"
    print("="*70)
    print("\033[1m\033[4mTURN 3\033[0m")
    print("="*70)
    print("\033[1mUser:\033[0m", message3)
    response3 = chat.send_message(message3)
    print("\033[1mAssistant:\033[0m", response3.text)
    
except Exception as e:
    # Handle errors (e.g., rate limits) and explain the concept
    print(f"⚠ Chat error: {str(e)[:150]}")
    print("\nNote: Chat sessions may hit rate limits. Here's how multi-turn conversations work:")
    print("\n1. Create a chat session with a config")
    print("\n2. Send messages using chat.send_message(message)")
    print("\n3. The model maintains context automatically across turns")
    print("\n4. Each response is based on the entire conversation history")    

## 7. Stream Responses

Stream responses in real-time to receive output progressively instead of waiting for the complete response.

### Language Modeling: Streaming and Token-by-Token Generation

**Streaming** reveals the fundamental nature of autoregressive language models: they generate text one token at a time.

**Autoregressive Generation Process:**
1. Given input tokens $[t_1, t_2, ..., t_n]$, predict $t_{n+1}$
2. Append $t_{n+1}$ to the sequence: $[t_1, t_2, ..., t_n, t_{n+1}]$
3. Use the expanded sequence to predict $t_{n+2}$
4. Repeat until a stopping condition (max tokens, end-of-sequence token, etc.)

**Why Streaming Matters:**
- **User Experience**: Users see output immediately instead of waiting for complete response
- **Latency**: Reduces perceived response time, especially for long outputs
- **Interactivity**: Enables real-time applications like chatbots and code assistants
- **Debugging**: Helps understand the model's generation process

**Technical Implementation:**
Instead of waiting for the entire forward pass to complete, the API sends each generated token (or small chunks) as they're produced. This requires maintaining the model state across multiple network responses.

**Trade-offs:**
- More network overhead (multiple requests vs. one)
- Slightly higher computational cost
- Better user experience for long responses

Let's observe streaming in action:

In [ ]:
import time  # For measuring response timing

# Configure the model and prompt
model = "gemini-2.5-flash"
prompt = "Write a story about a robot learning to dance. EXACTLY 7 sentences. No line breaks."

# Set generation parameters for creative output
config = types.GenerateContentConfig(
    max_output_tokens=1000,          # Allow long story
    temperature=0.9,                  # High creativity for storytelling
    thinking_config=thinking_config,  # Disable thinking tokens
)

print("\033[1m\033[4mStreaming Response:\033[0m")
print("=" * 70)

# Track timing information
t0 = time.time()
chunk_count = 0

# Stream the response token-by-token (or in small chunks)
for chunk in client.models.generate_content_stream(
    model=model,
    contents=prompt,
    config=config,
):
    chunk_count += 1
    dt = time.time() - t0  # Time elapsed since start

    # Extract text from the current chunk
    text_piece = chunk.text or ""

    # Display chunk metadata and content as it arrives
    print(f"\n[Chunk {chunk_count} @ {dt:.3f}s | {len(text_piece)} chars]")
    if text_piece:
        print(text_piece, end="", flush=True)  # Print without newline, flush immediately

## 8. Working with Different Multi-modal Contents

Gemini can handle multiple content types including text, images, and files. Let's demonstrate with text and image examples.

### Language Modeling: Multimodal Understanding

Modern LLMs like Gemini are **multimodal**, meaning they can process and understand multiple types of data: text, images, audio, and video.

**How Multimodal Models Work:**
1. **Unified Embedding Space**: Different modalities (text, images) are projected into a shared vector space
2. **Cross-modal Attention**: The model learns to attend to relevant features across modalities
3. **Joint Training**: Models are trained on paired data (e.g., image-caption pairs) to learn alignments

**Architecture:**
```
Image → Image Encoder (Vision Transformer) → Embeddings ─┐
                                                          ├→ Unified Transformer → Output
Text → Text Tokenizer → Embeddings ─────────────────────┘
```

**Applications:**
- Image captioning and description
- Visual question answering (VQA)
- Document understanding (OCR + comprehension)
- Code analysis from screenshots
- Meme interpretation

**Why This Matters:**
Traditional text-only LLMs are limited to linguistic information. Multimodal models can:
- Understand visual context
- Ground language in perceptual information
- Bridge the gap between symbolic (text) and perceptual (image) representations

Let's explore both text-based code analysis and visual understanding:

### 8.1 Text with Code Blocks

Gemini can analyze code and provide feedback.

In [ ]:
model = "gemini-2.5-flash"

# Sample code with intentional issues for the model to analyze
code_to_analyze = """
def fibonacci(n):
if n <= 1:
    return n
return fibonacci(n-1) + fibonacci(n-2)

result = fibonacci(10)
print(result)
"""

# Create a structured prompt for code review
prompt = f"""Review this Python code and suggest improvements:

```python
{code_to_analyze}
```

Focus on:
1. Performance issues
2. Code clarity
3. Best practices"""

# Generate code review and suggestions
response = client.models.generate_content(
    model=model,
    contents=prompt,
)

# Display the AI-generated code review
print("\033[1m\033[4mCode Review:\033[0m")
print(response.text)

### 8.2 Working with Images

Gemini's multimodal capabilities allow it to analyze and understand images alongside text. This example demonstrates **visual question answering (VQA)** - providing an image and a text prompt to get a description or answer.

**How Image Analysis Works:**
1. **Download/Load Image**: Images can come from URLs, local files, or be generated
2. **Convert to Bytes**: The image is converted to raw bytes format
3. **Create Part Object**: Using `types.Part.from_bytes()`, we wrap the image data with its MIME type
4. **Send Combined Request**: Both text prompt and image are sent together to the model
5. **Receive Analysis**: The model processes both modalities and returns text describing the image

**Supported Image Formats:**
- PNG, JPEG, WebP, HEIC, HEIF
- Maximum file size: 20MB (varies by model)
- Images are automatically resized if needed

**Use Cases:**
- Logo/brand identification
- Product recognition and description
- Document OCR and understanding
- Scene description for accessibility
- Medical image analysis
- Quality inspection

⚠️ **Important Note**: The `gemini-3-flash-preview` model used in this example is **NOT** available in the free tier. If you encounter quota errors or want to experiment with advanced models, you'll need to:
1. Go to [Google AI Studio](https://aistudio.google.com/)
2. Navigate to **Billing** settings
3. Add a payment method and upgrade your account
4. Check the [Pricing page](https://ai.google.dev/gemini-api/docs/pricing) for current rates

In [ ]:
from google import genai
from google.genai import types
import requests  # For downloading images from URLs
from PIL import Image  # For displaying images
from io import BytesIO

# Select a multimodal model that can process both text and images
model = "gemini-3-flash-preview"

# URL of the image to analyze
image_url = "https://www.google.com/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png"
prompt = "What is this logo? Describe it briefly."

# Download the image as bytes
image_bytes = requests.get(image_url, timeout=30).content

# Display the image in the notebook
print("\033[1m\033[4mImage Being Analyzed:\033[0m")
image = Image.open(BytesIO(image_bytes))
display(image)
print()

# Create a Part object from the image bytes
image_part = types.Part.from_bytes(
    data=image_bytes,
    mime_type="image/png",  # Specify the image format
)

try:
    # Send both text prompt and image to the model
    response = client.models.generate_content(
        model=model,
        contents=[prompt, image_part],  # Order can be: [prompt, image] or [image, prompt]
    )
    # Display the model's description of the image
    print("\033[1m\033[4mImage Analysis Result:\033[0m")
    print(response.text)
except Exception as e:
    # Handle any errors during image analysis
    print("\033[1m\033[4mError:\033[0m")
    print(repr(e))

### 8.3 Image Generation

This block demonstrates **text-to-image generation** using Gemini's multimodal output capabilities. This feature allows you to request both text descriptions and actual generated images in a single API call.

**How Image Generation Works:**
1. **Text Prompt**: Provide a detailed description of the image you want to generate
2. **Specify Response Modalities**: Set `response_modalities=[types.Modality.TEXT, types.Modality.IMAGE]`
3. **Model Processing**: The model generates both a text response and image data
4. **Extract Image Data**: Parse the response to find the inline image data
5. **Decode & Display**: Convert base64 data to an actual image file

**Best Practices for Prompts:**
- Be specific about style (e.g., "photorealistic", "watercolor", "studio ghibli style")
- Include details about composition, lighting, and mood
- Mention technical aspects (e.g., "ultra-detailed", "4K", "cinematic")
- Specify subject and background clearly

**Image Generation Parameters:**
- **Format**: Images are returned as base64-encoded data
- **Quality**: Depends on model and prompt specificity
- **Size**: Typically 1024x1024 or similar standard sizes
- **Speed**: May take longer than text-only generation

⚠️ **Important Note**: Again, the `gemini-3-pro-image-preview` model used in this example is **NOT** available in the free tier. If you encounter quota errors or want to experiment with advanced models, you'll need to:
1. Go to [Google AI Studio](https://aistudio.google.com/)
2. Navigate to **Billing** settings
3. Add a payment method and upgrade your account
4. Check the [Pricing page](https://ai.google.dev/gemini-api/docs/pricing) for current rates

In [ ]:
import base64              # For decoding base64-encoded image data

# Select a model that supports image generation
model_id = "gemini-3-pro-image-preview"  # Must use image-capable model

# Describe the image you want to generate
prompt_text = "A futuristic city skyline at sunset, ultra-detailed digital art, in studio ghibli style"

try:
    # Generate both text description and the actual image
    response = client.models.generate_content(
        model=model_id,
        contents=prompt_text,
        config=types.GenerateContentConfig(
            # Request both text and image in the response
            response_modalities=[types.Modality.TEXT, types.Modality.IMAGE]
        ),
    )

    # Display any text description the model provides
    if response.text:
        print("\033[1m\033[4mText Output:\033[0m")
        print(response.text)
        print()

    # Extract the image from the response parts
    image_data = None
    for part in response.candidates[0].content.parts:
        # Try multiple possible locations for image data
        if hasattr(part, 'inline_data') and part.inline_data:
            if hasattr(part.inline_data, 'data'):
                image_data = part.inline_data.data
                break
        elif hasattr(part, 'data'):
            image_data = part.data
            break

    if image_data:
        # Check if data is already bytes or needs base64 decoding
        if isinstance(image_data, bytes):
            image_bytes = image_data
        else:
            image_bytes = base64.b64decode(image_data)
        
        # Convert bytes to PIL Image
        image = Image.open(BytesIO(image_bytes))
        
        # Save to file
        image.save("gemini_generated.png")
        print("\033[1m\033[4mImage Status:\033[0m Saved as 'gemini_generated.png'")
        print()
        
        # Display the image in the notebook
        print("\033[1m\033[4mGenerated Image:\033[0m")
        display(image)
    else:
        print("\033[1m\033[4mNote:\033[0m Image generation may not be available for this model.")
        print("The model returned text only. Try using the model for text-to-image tasks via the web interface.")
        
except Exception as e:
    print("\033[1m\033[4mError:\033[0m", str(e))
    print("\nImage generation might not be available in the current API version.")
    print("The gemini-3-pro-image-preview model is primarily for image understanding, not generation.")
    print("For image generation, consider using dedicated image generation APIs like Imagen.")

## 9. Other Common Use Cases

Here are some practical examples of common use cases:

### 9.1 Content Summarization, Data Extraction, Question Answering

In [ ]:
# EXAMPLE 1: Content Summarization
print("\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mEXAMPLE 1: Content Summarization\033[0m")
print("\033[1m" + "="*70 + "\033[0m")

# Sample article to summarize
article = """
Machine learning is transforming how we solve complex problems.
From healthcare to finance, AI applications are improving efficiency
and enabling new discoveries. Recent breakthroughs in deep learning
have made systems capable of understanding language and images.
However, challenges remain in interpretability and bias detection.
"""

# Generate a concise summary
response = client.models.generate_content(
    model = "gemini-3-flash-preview",  # Fast model for summarization
    contents=f"Summarize this article in 2 sentences:\n\n{article}",
    config=types.GenerateContentConfig(max_output_tokens=100, thinking_config=thinking_config,)
)
print("\033[1m\033[4mSummary:\033[0m")
print(response.text)

# EXAMPLE 2: Data Extraction
print("\n" + "\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mEXAMPLE 2: Data Extraction\033[0m")
print("\033[1m" + "="*70 + "\033[0m")

# Unstructured text containing information to extract
text = "John Smith, age 32, works at TechCorp. Contact: john@example.com"

# Extract structured data as JSON
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""Extract structured data from this text and return as JSON:
    
{text}

Expected fields: name, age, company, email""",
    config=types.GenerateContentConfig(max_output_tokens=200, thinking_config=thinking_config,)
)
print("\033[1m\033[4mExtracted Data:\033[0m")
print(response.text)

# EXAMPLE 3: Question Answering
print("\n" + "\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mEXAMPLE 3: Question Answering\033[0m")
print("\033[1m" + "="*70 + "\033[0m")

# Context document for answering questions
context = """
The Great Wall of China is approximately 13,171 miles long.
It was built over many centuries to protect against invasions.
Construction began as early as the 7th century BC.
"""

# Answer a question based on the provided context
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"""Answer the question based on the context:

Context: {context}

Question: How long is the Great Wall of China?""",
    config=types.GenerateContentConfig(max_output_tokens=100, thinking_config=thinking_config,)
)
print("\033[1m\033[4mAnswer:\033[0m")
print(response.text)

### 9.2 Building a Mini Q&A System with Context

Let's build a simple question-answering system that demonstrates retrieval-augmented generation (RAG) concepts.

In [ ]:
model = "gemini-2.5-flash"

# Compare different prompting strategies for the same task
task = "Explain machine learning"

prompting_strategies = {
    "Vague": "Explain machine learning",
    
    "Specific with audience": "Explain machine learning to a 10-year-old child using simple analogies",
    
    "Structured output": """Explain machine learning with the following structure:
    1. Definition (1 sentence)
    2. How it works (2 sentences)
    3. Real-world example (1 sentence)
    4. Why it matters (1 sentence)""",
    
    "With constraints": """Explain machine learning in exactly 4 sentences. 
    Use the analogy of teaching a pet a trick. 
    Avoid technical jargon."""
}

print("\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mPROMPT ENGINEERING: Strategy Comparison\033[0m")
print("\033[1m" + "="*70 + "\033[0m")

for strategy_name, prompt in prompting_strategies.items():
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0.5,
            max_output_tokens=100,
            thinking_config=thinking_config,
        )
    )
    
    print(f"\n{'='*70}")
    print(f"\033[1m\033[4mSTRATEGY: {strategy_name}\033[0m")
    print(f"{'='*70}")
    print(f"\033[1mPrompt:\033[0m {prompt[:80]}..." if len(prompt) > 80 else f"\033[1mPrompt:\033[0m {prompt}")
    print(f"\n\033[1mResponse:\033[0m")
    print(response.text)
    print()

print("\n" + "=" * 70)
print("Notice how different prompting strategies yield different results!")
print("Key lessons:")
print("- Specificity improves relevance")
print("- Audience targeting adjusts complexity")
print("- Structure ensures completeness")
print("- Constraints control format and style")

### 9.3 Chain-of-Thought Reasoning

Demonstrates how prompting the model to think step-by-step improves accuracy on reasoning tasks.

In [ ]:
model = "gemini-2.5-flash"

# Complex reasoning problem
problem = """A farmer has 17 sheep, and all but 9 die. How many are left?

Let's approach this step-by-step:
1. Carefully read and understand what "all but 9" means
2. Break down the problem
3. Calculate the answer
4. Verify our reasoning

Think through this carefully before answering."""

response = client.models.generate_content(
    model=model,
    contents=problem,
    config=types.GenerateContentConfig(
        temperature=0.3,
        max_output_tokens=400,
        thinking_config=thinking_config,
    )
)

print("\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mCHAIN-OF-THOUGHT REASONING\033[0m")
print("\033[1m" + "="*70 + "\033[0m")
print(f"\033[1mProblem:\033[0m {problem.split('Let')[0].strip()}")
print()
print("\033[1mModel's Reasoning:\033[0m")
print(response.text)

### 9.4 Creative Writing: Story Generation with Style Transfer

LLMs can generate creative content in various styles by learning from the patterns and structures in their training data.

In [ ]:
model = "gemini-2.5-flash"

# Story prompt with style variations
base_story = "A programmer discovers their code has become sentient."

styles = [
    "Shakespearean drama",
    "Hard science fiction",
    "Children's picture book"
]

print("\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mCREATIVE WRITING: Style Transfer\033[0m")
print("\033[1m" + "="*70 + "\033[0m")
print(f"\033[1mBase Story Premise:\033[0m {base_story}")
print()

for idx, style in enumerate(styles, 1):
    prompt = f"""Write a short story (3-4 paragraphs) based on this premise:
"{base_story}"

Write it in the style of: {style}

Make it engaging and authentic to that style."""

    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0.9,  # High temperature for creativity
            max_output_tokens=500,
            thinking_config=thinking_config,
        )
    )
    
    print(f"\n{'='*70}")
    print(f"\033[1m\033[4mSTYLE {idx}: {style.upper()}\033[0m")
    print(f"{'='*70}")
    print(response.text)
    print()

### 9.5 Text Classification and Sentiment Analysis

LLMs excel at **zero-shot** and **few-shot learning**, where they can perform tasks without explicit training, leveraging their broad pre-training.

**Zero-shot Learning**: Performing a task with just a description, no examples.
**Few-shot Learning**: Performing a task with a handful of examples in the prompt.

In [ ]:
model = "gemini-2.5-flash"

# Zero-shot sentiment analysis
reviews = [
    "This product exceeded my expectations! Absolutely love it.",
    "Terrible quality, broke after one day. Do not buy.",
    "It's okay, nothing special but gets the job done.",
    "Best purchase I've made this year! Highly recommend."
]

prompt = """Analyze the sentiment of these customer reviews and classify each as POSITIVE, NEGATIVE, or NEUTRAL.
Also provide a confidence score (0-1) and a brief reason.

Format as JSON array with fields: review, sentiment, confidence, reason

Reviews:
"""

for i, review in enumerate(reviews, 1):
    prompt += f"{i}. {review}\n"

response = client.models.generate_content(
    model=model,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.1,  # Low temperature for consistent classification
        max_output_tokens=800,
        thinking_config=thinking_config,
    )
)

print("\033[1m" + "="*70 + "\033[0m")
print("\033[1m\033[4mSentiment Analysis Results:\033[0m")
print("\033[1m" + "="*70 + "\033[0m")
print(response.text)